In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from functions import *
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
# Define the file path
file_path = '/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/Shortest_Path_Matrix.csv'

# Read the CSV file into a DataFrame, specifying the separator as ';' and setting data types for index and headers
distances = (pd.read_csv(file_path, sep=';', index_col=0, dtype={'origins': str}))/1000

# Example of accessing shortest path between two GeoUID values
origin_geo_uid = '4620149'  # Ensure GeoUIDs are treated as strings
destination_geo_uid = '4620222'

# Find shortest path
try:
    shortest_path = distances.loc[origin_geo_uid, destination_geo_uid]
    print(f"Shortest path from GeoUID {origin_geo_uid} to GeoUID {destination_geo_uid}: {shortest_path}")
except KeyError:
    print("One or both of the GeoUIDs not found.")

In [ ]:
# Assuming df_shortest_paths is your existing DataFrame from Shortest_Path_Matrix.csv
file_path_shortest_paths = '/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/Shortest_Path_Matrix.csv'
df_shortest_paths = pd.read_csv(file_path_shortest_paths, sep=';', index_col=0, dtype={'origins': str})

# Extract all unique GeoUIDs from the shortest path DataFrame
geo_uids = df_shortest_paths.index.tolist()

# Example number of hours in a day
hours_experiment = 28  

# Create an empty DataFrame to store hourly passenger demand matrices
passenger_demand = {}

# Example of reading passenger demand files for each hour and filling empty_matrix
for hour in range(hours_experiment):
    # Example file path for passenger demand data for each hour
    file_path_demand = f'/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/24_1hour_od/car_{hour}h.csv'

    # Read passenger demand data for the current hour
    df_demand_hour = pd.read_csv(file_path_demand, sep=',', index_col=0, dtype={'origins': str})

    # Create an empty matrix for the current hour with float dtype
    empty_matrix = pd.DataFrame(0.0, index=geo_uids, columns=geo_uids)

    # Fill empty_matrix for the current hour
    for origin, row in df_demand_hour.iterrows():
        origin = str(origin)
        if origin.endswith('.0'):
            origin = origin[:-2]  # Remove the '.0' suffix if it exists
        for destination, demand in row.items():
            destination = str(destination)
            if destination.endswith('.0'):
                destination = destination[:-2]  # Remove the '.0' suffix if it exists
            if origin in empty_matrix.index and destination in empty_matrix.columns:
                if pd.notna(demand):
                    empty_matrix.at[origin, destination] = round(demand)
    
    # Store the filled DataFrame in the hourly_dfs dictionary
    passenger_demand[hour] = empty_matrix

# Iterate through the dictionary and print the non-zero values
for hour, df in passenger_demand.items():
    print(f"Non-zero values for hour {hour}:")
    non_zero_values = df[df != 0].stack()  # Stack to get a Series of non-zero values
    print(non_zero_values)


In [ ]:
# Iterate through the dictionary and calculate the sum of non-zero values
for hour, df in passenger_demand.items():
    non_zero_sum = df[df != 0].sum().sum()  # Sum all non-zero values
    print(f"Sum of non-zero values for hour {hour}: {non_zero_sum}")

In [ ]:
# Calculate the number of commutes per hour
commutes_per_hour = []

for hour, df in passenger_demand.items():
    # Count the number of non-zero entries in the DataFrame
    number_pasengers = df[df != 0].sum().sum()
    commutes_per_hour.append(number_pasengers)

# Plotting the number of commutes per hour
plt.figure(figsize=(10, 6))
plt.bar(range(hours_experiment), commutes_per_hour, color='blue')
plt.xlabel('Hour')
plt.ylabel('Passengers')
plt.title('Number of Passengers per Hour')
plt.xticks(range(hours_experiment))  # Set x-axis ticks to match hours
plt.grid(axis='y')
plt.show()

In [ ]:
def passenger_demand(file_path_shortest_paths='/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/Shortest_Path_Matrix.csv', hours_experiment=28, plot=True):
    # Assuming df_shortest_paths is your existing DataFrame from Shortest_Path_Matrix.csv
    df_shortest_paths = pd.read_csv(file_path_shortest_paths, sep=';', index_col=0, dtype={'origins': str})
    
    # Extract all unique GeoUIDs from the shortest path DataFrame
    geo_uids = df_shortest_paths.index.tolist()
    num_geo_uids = len(geo_uids)

    # Initialize a 3D NumPy array to store hourly passenger demand matrices
    passenger_demand = np.zeros((num_geo_uids, num_geo_uids, hours_experiment))

    # Example of reading passenger demand files for each hour and filling the array
    for hour in range(hours_experiment):
        # Example file path for passenger demand data for each hour
        file_path_demand = f'/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/24_1hour_od/car_{hour}h.csv'

        # Read passenger demand data for the current hour
        df_demand_hour = pd.read_csv(file_path_demand, sep=',', index_col=0, dtype={'origins': str})

        # Fill the array for the current hour
        for origin, row in df_demand_hour.iterrows():
            origin = str(origin)
            if origin.endswith('.0'):
                origin = origin[:-2]  # Remove the '.0' suffix if it exists
            for destination, demand in row.items():
                destination = str(destination)
                if destination.endswith('.0'):
                    destination = destination[:-2]  # Remove the '.0' suffix if it exists
                if pd.notna(demand):
                    if origin in geo_uids and destination in geo_uids:
                        origin_idx = geo_uids.index(origin)
                        destination_idx = geo_uids.index(destination)
                        passenger_demand[origin_idx, destination_idx, hour] = round(demand)

    if plot == True:
        # Calculate the total number of passengers per hour by summing all non-zero entries
        commutes_per_hour = []

        for hour in range(hours_experiment):
            # Sum all non-zero entries in the array for the current hour
            number_passengers = np.sum(passenger_demand[:, :, hour])
            commutes_per_hour.append(number_passengers)

        # Plotting the number of passengers per hour
        plt.figure(figsize=(10, 6))
        plt.bar(range(hours_experiment), commutes_per_hour, color='blue')
        plt.xlabel('Hour')
        plt.ylabel('Number of Passengers')
        plt.title('Number of Passengers per Hour')
        plt.xticks(range(hours_experiment))  # Set x-axis ticks to match hours
        plt.grid(axis='y')
        plt.show()
    
    return passenger_demand

# Example usage
passenger_demand= passenger_demand()

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

def coordinates(data='/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/CT_Centroids_with_YX.csv', plot=True, num_entries=None):
    df = pd.read_csv(data)
    
    # Limit the number of entries if specified
    if num_entries:
        df = df.head(num_entries)

    # Store the coordinates and GeoUID
    coordinates = df[['GeoUID', 'Y', 'X']]

    if plot:
        # Create a GeoDataFrame from the DataFrame
        gdf = gpd.GeoDataFrame(
            df, geometry=gpd.points_from_xy(df['X'], df['Y'])
        )

        # Plot the coordinates on a map
        fig, ax = plt.subplots(figsize=(10, 10))

        # Plot the points
        gdf.plot(ax=ax, color='blue', markersize=5)

        # Optionally, add GeoUID labels to the plot
        for x, y, label in zip(df['X'], df['Y'], df['GeoUID']):
            ax.text(x, y, label, fontsize=8, ha='right', va='bottom')

        # Set plot title and labels
        ax.set_title('Coordinates with GeoUIDs')
        ax.set_xlabel('Longitude (X)')
        ax.set_ylabel('Latitude (Y)')

    plt.show()
    return coordinates

def shortest_path(file_path='/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/Shortest_Path_Matrix.csv', num_entries=None):
    # Read the CSV file into a DataFrame, specifying the separator as ';' and setting data types for index and headers
    distances = pd.read_csv(file_path, sep=';', index_col=0, dtype={'origins': str}) / 1000
    
    # Limit the number of entries if specified
    if num_entries:
        distances = distances.head(num_entries)
        distances = distances.loc[:, distances.columns[:num_entries]]
    
    return distances

def passenger_demand(file_path_shortest_paths='/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/Shortest_Path_Matrix.csv', hours_experiment=28, plot=True, num_entries=None):
    # Read the CSV file into a DataFrame, specifying the separator as ';' and setting data types for index and headers
    df_shortest_paths = pd.read_csv(file_path_shortest_paths, sep=';', index_col=0, dtype={'origins': str})
    
    # Limit the number of entries if specified
    if num_entries:
        df_shortest_paths = df_shortest_paths.head(num_entries)
        df_shortest_paths = df_shortest_paths.loc[:, df_shortest_paths.columns[:num_entries]]
    
    # Extract all unique GeoUIDs from the shortest path DataFrame
    geo_uids = df_shortest_paths.index.tolist()
    num_geo_uids = len(geo_uids)

    # Initialize a 3D NumPy array to store hourly passenger demand matrices
    passenger_demand = np.zeros((num_geo_uids, num_geo_uids, hours_experiment))

    # Example of reading passenger demand files for each hour and filling the array
    for hour in range(hours_experiment):
        # Example file path for passenger demand data for each hour
        file_path_demand = f'/Users/natomanzolli/Documents/PhD/Artigos/Journals/SAEVs paper/Origin_destination_matrices/24_1hour_od/car_{hour}h.csv'

        # Read passenger demand data for the current hour
        df_demand_hour = pd.read_csv(file_path_demand, sep=',', index_col=0, dtype={'origins': str})
        
        if num_entries:
            df_demand_hour = df_demand_hour.head(num_entries)
            df_demand_hour = df_demand_hour.loc[:, df_demand_hour.columns[:num_entries]]

        # Fill the array for the current hour
        for origin, row in df_demand_hour.iterrows():
            origin = str(origin)
            if origin.endswith('.0'):
                origin = origin[:-2]  # Remove the '.0' suffix if it exists
            for destination, demand in row.items():
                destination = str(destination)
                if destination.endswith('.0'):
                    destination = destination[:-2]  # Remove the '.0' suffix if it exists
                if pd.notna(demand):
                    if origin in geo_uids and destination in geo_uids:
                        origin_idx = geo_uids.index(origin)
                        destination_idx = geo_uids.index(destination)
                        passenger_demand[origin_idx, destination_idx, hour] = round(demand)

    if plot:
        # Calculate the total number of passengers per hour by summing all non-zero entries
        commutes_per_hour = []

        for hour in range(hours_experiment):
            # Sum all non-zero entries in the array for the current hour
            number_passengers = np.sum(passenger_demand[:, :, hour])
            commutes_per_hour.append(number_passengers)

        # Plotting the number of passengers per hour
        plt.figure(figsize=(10, 6))
        plt.bar(range(hours_experiment), commutes_per_hour, color='blue')
        plt.xlabel('Hour')
        plt.ylabel('Number of Passengers')
        plt.title('Number of Passengers per Hour')
        plt.xticks(range(hours_experiment))  # Set x-axis ticks to match hours
        plt.grid(axis='y')
        plt.show()
    
    return passenger_demand

In [3]:
import os
import pandas as pd

def read_and_combine_excel_sheets(folder_path, sheet_name, output_file):
    # List to hold dataframes
    data_frames = []

    # Loop through all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xlsx"):
            # Construct full file path
            file_path = os.path.join(folder_path, file_name)
            
            # Read the specific sheet from the excel file
            try:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                # Add a column for the file name
                df['Source_File'] = file_name
                data_frames.append(df)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

    # Combine all dataframes into one
    combined_df = pd.concat(data_frames, ignore_index=True)
    
    # Write the combined dataframe to a new Excel file
    combined_df.to_excel(output_file, index=False)

# Example usage
folder_path = '/Users/natomanzolli/Downloads/SAEVs'
sheet_name = 'Objective_Function_Values'
output_file = 'combined_output.xlsx'

read_and_combine_excel_sheets(folder_path, sheet_name, output_file)


In [16]:
import pandas as pd

def sum_first_four(values):
    result = []
    index = 0
    while index < len(values):
        # Take up to the next 4 elements
        chunk = values[index:index + 4]
        # Sum the elements in the chunk
        chunk_sum = sum(chunk)
        # Append the sum to the result list
        result.append(chunk_sum)
        # Move to the next set of 4 elements
        index += 4
    return result

# New list of numbers provided
numbers = [
    41.5, 41.5, 41.5, 41.5, 100, 100, 100, 100, 235.75, 235.75, 235.75, 235.75,
    755.25, 755.25, 755.25, 755.25, 2460.25, 2460.25, 2460.25, 2460.25, 5302.25,
    5302.25, 5302.25, 5302.25, 7391, 7391, 7391, 7391, 8958.5, 8958.5, 8958.5,
    8958.5, 8686.75, 8686.75, 8686.75, 8686.75, 5978.5, 5978.5, 5978.5, 5978.5,
    4593.75, 4593.75, 4593.75, 4593.75, 3860.5, 3860.5, 3860.5, 3860.5, 3760.5,
    3760.5, 3760.5, 3760.5, 3817, 3817, 3817, 3817, 3455.25, 3455.25, 3455.25,
    3455.25, 2811.75, 2811.75, 2811.75, 2811.75, 8717, 8717, 8717, 8717, 7626.25,
    7626.25, 7626.25, 7626.25, 6166.5, 6166.5, 6166.5, 6166.5, 5350, 5350, 5350,
    5350, 3595.5, 3595.5, 3595.5, 3595.5, 2968, 2968, 2968, 1484, 1446.5, 1446.5,
    723.25, 723.25, 334.5, 334.5, 334.5, 334.5
]

# Calculate the sums of every 4 elements
sums = sum_first_four(numbers)

# Create a DataFrame from the result
df = pd.DataFrame(sums, columns=['Sum of 4 Values'])

# Save the DataFrame to an Excel file
df.to_excel('sums.xlsx', index=False)

print("Data has been successfully written to 'sums.xlsx'")



Data has been successfully written to 'sums.xlsx'


In [18]:
import pandas as pd

def repeat_elements(values):
    result = []
    for value in values:
        # Repeat each element 4 times and add to the result list
        value = value/4
        result.extend([value] * 4)
    return result

# List of numbers provided
numbers = [
    166, 400, 943, 3021, 9841, 21209, 29564, 35834, 34747, 23914,
    18375, 15442, 15042, 15268, 13821, 11247, 30505, 34868, 24666,
    21400, 14382, 10388, 4339.5, 1338
]

# Generate the new list with each element repeated 4 times
repeated_list = repeat_elements(numbers)

# Create a DataFrame from the repeated list
df_repeated = pd.DataFrame(repeated_list, columns=['Repeated Values'])

# Save the DataFrame to an Excel file
df_repeated.to_excel('repeated_values.xlsx', index=False)

print("Data has been successfully written to 'repeated_values.xlsx'")


Data has been successfully written to 'repeated_values.xlsx'


In [4]:
import pandas as pd
import plotly.graph_objects as go

# Load the Excel file
file_path = 'sensitivity_analysis.xlsx'
df = pd.read_excel(file_path, index_col=0)

# Convert all data to numeric, coercing errors and replacing NaNs with 0
df = df.apply(pd.to_numeric, errors='coerce').fillna(0)

# Define the categories (rows) and the scenarios (columns)
categories = df.index
scenarios = df.columns

# Create a radar plot for each scenario
fig = go.Figure()

for scenario in scenarios:
    fig.add_trace(go.Scatterpolar(
        r=df[scenario],
        theta=categories,
        fill='toself',
        name=scenario
    ))

# Update the layout of the plot
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, df.max().max()]  # Maximum value after conversion
        )),
    showlegend=True,
    title="Sensitivity Analysis Radar Plot"
)

# Show the plot
fig.show()
